In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist= input_data.read_data_sets("MNIST_data",one_hot=True)

#every batch of size
batch_size=100

n_batch= mnist.train.num_examples // batch_size
print(n_batch)
#initialize weight
def weight_variable(shape):
    initial= tf.truncated_normal(shape,stddev=0.1)#正态分布
    return tf.Variable(initial)
#initialize bias
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    #Computes a 2-D convolution given 4-D input and filter tensors
    # x: Flattens the filter to a 2-D matrix with shape
    #   [filter_height * filter_width * in_channels, output_channels].
    # W :Extracts image patches from the input tensor to form a virtual tensor of shape 
    #    [batch, out_height, out_width, filter_height * filter_width * in_channels].
    #strides[0]=strides[3]=1 步长
    #padding : SAME or VALID
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层

def max_pool_2x2(x):
    #kize[1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1,],padding='SAME')


x=tf.placeholder(tf.float32,[None,784])#28*28
y=tf.placeholder(tf.float32,[None,10])

#复原 to 4d 
x_image=tf.reshape(x,[-1,28,28,1])
#initialize convl 
W_conv1 = weight_variable([5,5,1,32]) #5x5 windows, 32convl from 1 image
#第三个数黑白为1，彩色为3 
b_conv1 = bias_variable([32])# bias

#把x_image和权值卷积，加上bias， 然后用relu激活
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1= max_pool_2x2(h_conv1) # 进行max_pool

W_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])

#把x_image和权值卷积，加上bias， 然后用relu激活
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2= max_pool_2x2(h_conv2) # 进行max_pool


#the first process from 28*28 to 14*14
#the second process from 14*14 to 7*7
#So, the result is 64 images by 7*7 size

#initialize 全连接层
W_fcl=weight_variable([7*7*64,1024])
b_fcl= bias_variable([1024])

#2D to 1D
h_pool2_flat= tf.reshape(h_pool2,[-1,7*7*64])
h_fcl=tf.nn.relu(tf.matmul(h_pool2_flat,W_fcl)+b_fcl)

#keep_prob 表示神经元输出概率
keep_prob=tf.placeholder(tf.float32)
h_fcl_drop = tf.nn.dropout(h_fcl,keep_prob)

W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])
#计算输出
prediction =tf.nn.softmax(tf.matmul(h_fcl_drop,W_fc2)+b_fc2)
#交叉跌代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
train_step =tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction= tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))


accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            if batch%100==0:
                print("trainning!")
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
            
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter="+str(epoch)+" ,Testing Accuracy="+str(acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
550
trainning!


KeyboardInterrupt: 